In [15]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import fitz
import PyPDF2

if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

#### Features to use : 
- Index of industrial production (%)
- Production of wood (Thous. m3)
- Poultry population (Mill. heads) (so suc vat trong nuoc suc vat)
- Livestock population (Thous. heads)
- Number of farms
- Production of cereals (Thous. tons)
- ENTERPRISE,COOPERATIVE AND NON-FARM INDIVIDUAL BUSINESS ESTABLISHMENT
- Area of floors of residential buildings constructed in the year by types of house (Thous. m2)
- Investment at current prices (Bill. dongs)
- Structure of investment at current prices (%)
- Expenditure on social and economic services
- STRUCTURE OF GRDP AT CURRENT PRICES (%)
- POPULATION DENSITY (Person/km2)
- POPULATION (Thous. pers.)
- AREA OF LAND (Thous. ha) 

In [29]:
features = [
    "Index of industrial production (%)",
    "Production of wood (Thous. m3)",
    "Poultry population (Mill. heads) (so suc vat trong nuoc suc vat)",
    "Livestock population (Thous. heads)",
    "Number of farms",
    "Production of cereals (Thous. tons)",
    "ENTERPRISE, COOPERATIVE AND NON-FARM INDIVIDUAL BUSINESS ESTABLISHMENT",
    "Area of floors of residential buildings constructed in the year by types of house (Thous. m2)",
    "Investment at current prices (Bill. dongs)",
    "Structure of investment at current prices (%)",
    "Expenditure on social and economic services",
    "STRUCTURE OF GRDP AT CURRENT PRICES (%)",
    "POPULATION DENSITY (Person/km2)",
    "POPULATION (Thous. pers.)",
    "AREA OF LAND (Thous. ha)"
]

econpdf = '../data/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf'

# Function to extract features pages and save them to the output folder
def extract_pages_and_save(pdf_path, features):
    doc = fitz.open(pdf_path)

    output_folder = '../data/output'
    os.makedirs(output_folder, exist_ok=True)

    # loop qua pages tim features
    for page_num in range(len(doc)):
        page = doc[page_num]
        page_text = page.get_text()
        
        # check pages which contains features 
        if any(feature in page_text for feature in features):
            # Save to the output folder
            output_file = os.path.join(output_folder, f'page_{page_num + 1}.pdf')
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
            new_doc.save(output_file)
            new_doc.close()

    doc.close()
# call function thu cai 
extract_pages_and_save(econpdf, features)


In [24]:

# def read_large_pdf(file_path):
#     pdf_document = fitz.open(file_path)

#     for page_number in range(pdf_document.page_count):
#         page = pdf_document[page_number]
#         text = page.get_text()
#         # Process the text or perform other operations as needed
#         print(f"Page {page_number + 1}: {text[:100]}...")  # Print the first 100 characters of each page

#     pdf_document.close()

# # open the pdf file
# econ_pdf = "../data/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf"
# read_large_pdf(econ_pdf)


In [25]:
# def remove_pages(input_path, output_path, start_page, end_page):
#     with open(input_path, 'rb') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         pdf_writer = PyPDF2.PdfWriter()

#         for page_number in range(len(pdf_reader.pages)):
#             if start_page <= page_number + 1 <= end_page:
#                 continue
#             pdf_writer.add_page(pdf_reader.pages[page_number])

#         with open(output_path, 'wb') as output_file:
#             pdf_writer.write(output_file)

# # Example usage: remove pages 1 to 1052
# remove_pages('../data/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf', '../data/econ_reduced.pdf', 1, 1052)
